# afspraken voor de input

Input voor de setpoint  = 750 - 900 oftevel 150 mm

Input voor de hoogte    = 200 tot 1000 oftewel 800 mm

Er kan gemeten worden tussen 200 tot 1000mm. Er wordt het beste gemeten tussen 750 en 900mm.



# Importeren Packages

In [24]:

import scipy.io
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import torch


# Functies defineren

In [25]:
def initialise(a, b):
    epsilon = 0.15
    # Maakt een lege tensor van grootte (a, b+1) en vult die met waarden uit uniform(-epsilon, +epsilon)
    c = torch.empty(a, b + 1).uniform_(-epsilon, epsilon)
    return c

In [ ]:
def neural_network(nn_params, input_layer_size, hidden_layer_size, output_layer_size, X, y, lamb):
    # Weights are split back to Theta1, Theta2
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, input_layer_size + 1))
    Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                        (output_layer_size, hidden_layer_size + 1))

    # Forward propagation
    m = X.shape[0]
    one_matrix = np.ones((m, 1))
    X = np.append(one_matrix, X, axis=1)  # Adding bias unit to first layer
    a1 = X
    z2 = np.dot(X, Theta1.transpose())
    a2 = 1 / (1 + np.exp(-z2))  # Activation for second layer
    one_matrix = np.ones((m, 1))
    a2 = np.append(one_matrix, a2, axis=1)  # Adding bias unit to hidden layer
    z3 = np.dot(a2, Theta2.transpose())
    a3 = 1 / (1 + np.exp(-z3))  # Activation for third layer

    # Changing the y labels into vectors of boolean values.
    # For each label between 0 and 9, there will be a vector of length 10
    # where the ith element will be 1 if the label equals i
    y_vect = np.zeros((m, 10))
    for i in range(m):
        y_vect[i, int(y[i])] = 1

    # Calculating cost function
    J = (1 / m) * (np.sum(np.sum(-y_vect * np.log(a3) - (1 - y_vect) * np.log(1 - a3)))) + (lamb / (2 * m)) * (
                sum(sum(pow(Theta1[:, 1:], 2))) + sum(sum(pow(Theta2[:, 1:], 2))))

    # backprop
    Delta3 = a3 - y_vect
    Delta2 = np.dot(Delta3, Theta2) * a2 * (1 - a2)
    Delta2 = Delta2[:, 1:]

    # gradient
    Theta1[:, 0] = 0
    Theta1_grad = (1 / m) * np.dot(Delta2.transpose(), a1) + (lamb / m) * Theta1
    Theta2[:, 0] = 0
    Theta2_grad = (1 / m) * np.dot(Delta3.transpose(), a2) + (lamb / m) * Theta2
    grad = np.concatenate((Theta1_grad.flatten(), Theta2_grad.flatten()))

    return J, grad


In [ ]:
def predict(Theta1, Theta2, X):
    m = X.shape[0]
    
    one_matrix = np.ones((m, 1))
    X = np.append(one_matrix, X, axis=1)  # Adding bias unit to first layer
    z2 = np.dot(X, Theta1.transpose())
    a2 = 1 / (1 + np.exp(-z2))  # Activation for second layer
    one_matrix = np.ones((m, 1))
    a2 = np.append(one_matrix, a2, axis=1)  # Adding bias unit to hidden layer
    z3 = np.dot(a2, Theta2.transpose())
    a3 = 1 / (1 + np.exp(-z3))  # Activation for third layer
    p = (np.argmax(a3, axis=1))  # Predicting the class on the basis of max value of hypothesis
    return p


# CSV Data importeren

In [29]:
csv_path = '/Users/mitchelreints/Documents/Visuel Studio Code/Workspace [Python]/#AIregelsysteem/simulatie_resultaten_2025-03-04_12-48-39.csv'
data = pd.read_csv(csv_path)
print(data)

      Tijd (s)  PWM (%)  Doel Hoogte (m)  Daadwerkelijke Hoogte (m)  \
0         0.01   100.00            0.500                      0.003   
1         0.02   100.00            0.500                      0.008   
2         0.03   100.00            0.500                      0.016   
3         0.04   100.00            0.500                      0.027   
4         0.05   100.00            0.500                      0.041   
...        ...      ...              ...                        ...   
996       9.97     0.00            0.576                      0.581   
997       9.98    68.37            0.576                      0.581   
998       9.99     0.00            0.576                      0.581   
999      10.00     0.00            0.576                      0.580   
1000     10.01   100.00            0.576                      0.581   

      Snelheid (m/s)  
0              0.272  
1              0.545  
2              0.817  
3              1.089  
4              1.361  
...      

## Data uit de csv halen

In [28]:
# Normaliseer de kolommen: Doel door 150, Daadwerkelijke door 800
X = data[["Doel Hoogte (m)", "Daadwerkelijke Hoogte (m)"]]

print(X)

X = X.transpose()
print(X)

m = X.shape[0]



# Labels
Y = data["PWM (%)"]
print(Y)
Y = Y.flatten()
print(Y)


      Doel Hoogte (m)  Daadwerkelijke Hoogte (m)
0               0.500                      0.003
1               0.500                      0.008
2               0.500                      0.016
3               0.500                      0.027
4               0.500                      0.041
...               ...                        ...
996             0.576                      0.581
997             0.576                      0.581
998             0.576                      0.581
999             0.576                      0.580
1000            0.576                      0.581

[1001 rows x 2 columns]
                            0      1      2      3      4      5      6     \
Doel Hoogte (m)            0.500  0.500  0.500  0.500  0.500  0.500  0.500   
Daadwerkelijke Hoogte (m)  0.003  0.008  0.016  0.027  0.041  0.057  0.076   

                            7      8      9     ...   991    992    993   \
Doel Hoogte (m)            0.500  0.500  0.500  ...  0.576  0.576  0.576   


AttributeError: 'Series' object has no attribute 'flatten'

# Labels uit de CSV halen

In [117]:
#Labels
Y = data["PWM (%)"].values
print(Y)
Y = Y.flatten()
print(Y)

[100. 100. 100. ...   0.   0. 100.]
[100. 100. 100. ...   0.   0. 100.]


## Normaliseren van de data

In [116]:
X.loc["Doel Hoogte (m)"] = X.loc["Doel Hoogte (m)"] / 1
X.loc["Daadwerkelijke Hoogte (m)"] = X.loc["Daadwerkelijke Hoogte (m)"] / 1
print(X)

                            0      1      2      3      4      5      6     \
Doel Hoogte (m)            0.500  0.500  0.500  0.500  0.500  0.500  0.500   
Daadwerkelijke Hoogte (m)  0.003  0.008  0.016  0.027  0.041  0.057  0.076   

                            7      8      9     ...   991    992    993   \
Doel Hoogte (m)            0.500  0.500  0.500  ...  0.576  0.576  0.576   
Daadwerkelijke Hoogte (m)  0.098  0.119  0.139  ...  0.581  0.582  0.582   

                            994    995    996    997    998    999    1000  
Doel Hoogte (m)            0.576  0.576  0.576  0.576  0.576  0.576  0.576  
Daadwerkelijke Hoogte (m)  0.581  0.581  0.581  0.581  0.581  0.580  0.581  

[2 rows x 1001 columns]


## Data splitten tussen training data met 500 voorbeelden

In [ ]:
X_train = X[:500, :]
Y_train = Y[:500]
print(X_train)
print(Y_train)

## Data splitten tussen test data met 500 voorbeelden

In [ ]:
X_test = X[500:, :]
Y_test = Y[500:]
print(X_test)
print(Y_test)

## Bepalen grootte neuraal netwerk

In [14]:
input_layer_size = 2  #Setpoints + Sample huidige hoogte + vorige Sample (N-1) + (N-2) + (N-3) + (N-...)
hidden_layer_size = 100
output_layer_size = 101 # 0 - 100 

## Random initializeren van de Thetas

In [19]:
initial_Theta1 = initialise(hidden_layer_size, input_layer_size)
initial_Theta2 = initialise(output_layer_size, hidden_layer_size)
print(initial_Theta1)
print(initial_Theta2)

tensor([[-0.0663,  0.0611, -0.0357],
        [ 0.0387, -0.0139, -0.0055],
        [ 0.1354,  0.0040,  0.0340],
        [-0.0164, -0.0765, -0.1105],
        [-0.1334, -0.1480, -0.0741],
        [ 0.1358,  0.1162, -0.0974],
        [ 0.0237,  0.1407,  0.0660],
        [-0.0906,  0.1099,  0.0446],
        [-0.1147,  0.1362,  0.0325],
        [ 0.0024,  0.1191, -0.0686],
        [ 0.0005, -0.1081,  0.1076],
        [ 0.0471, -0.0041, -0.0987],
        [-0.1112,  0.0760, -0.0385],
        [ 0.0494, -0.0837,  0.1115],
        [ 0.0739, -0.0400,  0.0903],
        [ 0.0484,  0.0799,  0.0221],
        [-0.1001,  0.0454, -0.0973],
        [-0.0079, -0.0719,  0.0744],
        [ 0.0559,  0.0810, -0.0543],
        [ 0.0078,  0.0837,  0.0424],
        [ 0.1135,  0.0838,  0.0702],
        [-0.0326, -0.0380,  0.1258],
        [ 0.0452, -0.1296,  0.1126],
        [ 0.0694, -0.0422,  0.0314],
        [ 0.1423, -0.0963, -0.0647],
        [ 0.0738, -0.0667,  0.0164],
        [-0.0288,  0.1405,  0.0670],
 

## Zet twee arrays bij elkaar

In [ ]:
initial_nn_params = np.concatenate((initial_Theta1.flatten(), initial_Theta2.flatten()))
print(initial_nn_params)

## Minimize fucntie aan roepen om de cost functie te minimaliseren en om de gewichten te trainen

In [119]:
maxiter = 100
lambda_reg = 0.1  # To avoid overfitting
myargs = (input_layer_size, hidden_layer_size, output_layer_size, X_train, Y_train, lambda_reg)

results = minimize(neural_network, x0=initial_nn_params, args=myargs,
          options={'disp': True, 'maxiter': maxiter}, method="L-BFGS-B", jac=True)
nn_params = results["x"]  # Trained Theta is extracted

# Assuming the correct shape should be (101, 101)
correct_shape = (101, 101)

# Check the size of nn_params
print("Size of nn_params:", nn_params.size)

# If the size matches the target shape, reshape it
if nn_params.size == correct_shape[0] * correct_shape[1]:
    reshaped_params = nn_params.reshape(correct_shape)
    print("Reshaped nn_params:", reshaped_params)
else:
    print(f"Cannot reshape array of size {nn_params.size} into shape {correct_shape}")



ValueError: cannot reshape array of size 10601 into shape (101,101)

## Gewichten worden verdeeld tussen de neuronen

In [118]:
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)], (
                              hidden_layer_size, input_layer_size + 1))  # Connecties tussen inputlayer en hidden_layer
Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                      (output_layer_size, hidden_layer_size + 1))  # connecties tussen hidden_layer en output_layer

NameError: name 'nn_params' is not defined

# Test Nauwkeurigheid checken

In [ ]:
pred = predict(Theta1, Theta2, X_test)
print('Test Nauwkeurigheid: {:f}'.format((np.mean(pred == y_test) * 100)))

# Training set nauwkeurigheid checken

In [ ]:
pred = predict(Theta1, Theta2, X_train)
print('Training Set Nauwkeurigheid: {:f}'.format((np.mean(pred == y_train) * 100)))